In [2]:
import os
print(os.environ['LD_LIBRARY_PATH'])
!echo $LD_LIBRARY_PATH

/usr/lib/R/lib:/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
/usr/lib/R/lib:/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server


In [3]:
# Import modules
import tensorflow as tf
from tensorflow import keras as K
from keras.models import load_model
import pandas as pd
import numpy as np
import PIL
import logging
import os

import zipfile
import shutil

# for figures
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns

# for the Grad-CAMs
from IPython.display import Image, display
import matplotlib.cm as cm

#Working Directory on A4 Computer (GoPro and V2 data are on the A4 Computer)

files = os.listdir()
    
os.chdir("/home/scottweichenthal/Dropbox/IMAGINE Project/MSSI_Project/")
cwd = os.getcwd()
print("Current working directory:", cwd)


#This code allows GPU memory allocation to grow as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    

Current working directory: /home/scottweichenthal/Dropbox/IMAGINE Project/MSSI_Project


2024-05-13 17:14:12.864559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-13 17:14:12.864909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-13 17:14:12.865211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-13 17:14:12.865511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-13 17:14:12.892144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [4]:
#  Prints the Tensorflow, Keras versions, and number of GPUs
print(tf.__version__)
print(K.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.7.0
2.7.0
Num GPUs Available:  4


In [5]:
#Which Python Version
from platform import python_version

print(python_version())

3.9.7


In [6]:
# Read metadata for compiled data you want to use for CNN model development
#This version has GoPro in trn and val sets
#metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_15012024.csv", low_memory=False)

#This version has GoPro data only in trn set (as opposed to random split between trn,val,tst)
# metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_04042024.csv", low_memory=False)
# Train: 100% GoPro + 80% Imagine V2
# Val: 10% Imagine V2
# Tst: 10% Imagine V2
# External Test: 100% Imagine V3


#This compiled file incorporate all existing data with GoPro in training set only. Rishabh Please use this file!

metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_04252024.csv", low_memory=False)
# Train: 100% GoPro + 80% Imagine V2 and V3
# Val: 10% Imagine V2 and V3
# Tst: 10% Imagine V2 and V3

In [ ]:
# Look at data
metadata.head(10)

In [8]:
#Look at variable names
for col in metadata.columns:
    print(col)

datetime
image_path
audio_path
ln_ufp_num_10s_ma_image_label_raw
ln_ufp_num_10s_ma_spec_label_raw
ln_ufp_size_10s_ma_image_label_raw
ln_ufp_size_10s_ma_spec_label_raw
ln_noise_10s_ma_image_label_raw
ln_noise_10s_ma_spec_label_raw
ufp_num_10s_ma_image_label_raw
ufp_num_10s_ma_spec_label_raw
ufp_size_10s_ma_image_label_raw
ufp_size_10s_ma_spec_label_raw
noise_10s_ma_image_label_raw
noise_10s_ma_spec_label_raw
temp_airp
wspd_airp
year
city
pm25
file_exists
hardware
site_id
device_id
file
noise_10s_ma_spec_label_quartile
noise_10s_ma_image_label_quartile
ufp_num_10s_ma_spec_label_quartile
ufp_num_10s_ma_image_label_quartile
ufp_size_10s_ma_spec_label_quartile
ufp_size_10s_ma_image_label_quartile
image_extension
audio_extension
image_name
audio_name
pair_pm25
set_V3ext
set
vsby_nm


In [15]:
# How much data in trn, val, tst sets for metadata_random_split

metadata.set.value_counts()

trn    345680
val     22407
tst     22223
Name: set, dtype: int64

In [16]:
metadata.describe()

,ln_ufp_num_10s_ma_image_label_raw,ln_ufp_num_10s_ma_spec_label_raw,ln_ufp_size_10s_ma_image_label_raw,ln_ufp_size_10s_ma_spec_label_raw,ln_noise_10s_ma_image_label_raw,ln_noise_10s_ma_spec_label_raw,ufp_num_10s_ma_image_label_raw,ufp_num_10s_ma_spec_label_raw,ufp_size_10s_ma_image_label_raw,ufp_size_10s_ma_spec_label_raw,...,pm25,device_id,noise_10s_ma_spec_label_quartile,noise_10s_ma_image_label_quartile,ufp_num_10s_ma_spec_label_quartile,ufp_num_10s_ma_image_label_quartile,ufp_size_10s_ma_spec_label_quartile,ufp_size_10s_ma_image_label_quartile,pair_pm25,vsby_nm
count,390310.000000,390310.000000,390310.000000,390310.000000,390310.000000,390310.000000,3.903100e+05,3.903100e+05,390310.000000,390310.000000,...,359005.000000,223147.000000,223147.000000,223147.000000,223147.000000,223147.000000,223147.000000,223147.000000,223147.000000,390039.000000
mean,9.301744,9.302244,3.513263,3.513208,4.115388,4.115988,1.512980e+04,1.513813e+04,36.045195,36.040962,...,6.629045,28.877045,1.446513,1.429632,1.481508,1.479643,1.554374,1.555392,6.911002,21.609274
std,0.779956,0.779607,0.385172,0.384935,0.118462,0.119192,2.221654e+04,2.205105e+04,13.837587,13.835993,...,3.997787,5.682456,1.124708,1.114186,1.118695,1.118875,1.129159,1.129161,6.502780,8.166993
min,2.895912,2.906901,2.302585,2.302585,3.598134,3.598134,1.810000e+01,1.830000e+01,10.000000,10.000000,...,0.666429,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.356000,0.380000
25%,8.812397,8.812073,3.255401,3.255015,4.041471,4.040768,6.717000e+03,6.714825e+03,25.930000,25.920000,...,3.626583,28.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.214500,15.000000
50%,9.317678,9.317849,3.564732,3.563883,4.121798,4.122608,1.113310e+04,1.113500e+04,35.330000,35.300000,...,6.105735,29.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,4.799500,15.000000
75%,9.831824,9.832088,3.790985,3.790985,4.197052,4.199455,1.861688e+04,1.862180e+04,44.300000,44.300000,...,8.358984,30.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,8.310000,30.000000
max,15.074470,14.446425,5.703782,5.703782,4.618876,4.627910,3.521756e+06,1.879329e+06,300.000000,300.000000,...,46.562680,41.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,80.094250,60.000000


In [17]:
#Select input file type (this tells python where to look for the file paths)
file = input("What input are you using: images, or spectrograms? ")

if file == 'images':
    file = 'image_path'
else:
  if file == 'spectrograms':
      file = 'audio_path'
  else:
    print('!!!TYPO in input_data name')

What input are you using: images, or spectrograms? images


In [18]:
# Define Initial Learning Rate
initial_learning_rate = 1e-4
meta_data_name = "gp_v2_v3_Training_Only"

In [19]:
architecture = input("What architecture do you want to use: Xception_linear_Nadam, ResNet50_linear_Nadam ?")
if architecture == 'Xception_linear_Nadam':
    architecture_preprocessing = K.applications.xception.preprocess_input
    def get_compiled_model():
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.Xception(include_top=False, weights = "imagenet", input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units= 1, activation='linear')(model_output) 
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=K.optimizers.Nadam(learning_rate = initial_learning_rate),
            loss = 'mse',
            metrics = ['mae']
        )
        return model
                    
elif architecture == 'ResNet50_linear_Nadam':
    architecture_preprocessing = K.applications.resnet50.preprocess_input
    def get_compiled_model():
        model_input = K.layers.Input(shape=(256, 256, 3), dtype='float32', name='input')
        conv_base = K.applications.ResNet50(include_top=False, weights= "imagenet", input_tensor=model_input)
        model_output = K.layers.GlobalAveragePooling2D()(conv_base.output)
        model_output = K.layers.Dense(units= 1, activation='linear')(model_output) 
        model = K.models.Model(inputs=model_input, outputs=model_output)
        model.compile(
            optimizer=K.optimizers.Nadam(learning_rate = initial_learning_rate),
            loss = 'mse',
            metrics = ['mae']
        )
        return model
                      
else:
    print('!!!TYPO in architecture name')

What architecture do you want to use: Xception_linear_Nadam, ResNet50_linear_Nadam ?ResNet50_linear_Nadam


In [20]:
# Select Exposure to be modelled
cwd = os.getcwd()
print("Current working directory:", cwd)

target = input("What do you want to model: ln_noise_10s_ma_spec_label_raw, ln_noise_10s_ma_image_label_raw, ln_ufp_num_10s_ma_spec_label_raw, ln_ufp_num_10s_ma_image_label_raw, ln_ufp_size_10s_ma_image_label_raw, ln_ufp_size_10s_ma_spec_label_raw ?")                  

if target == 'ln_noise_10s_ma_image_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.hdf5', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)


elif target == 'ln_noise_10s_ma_spec_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.hdf5', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    
elif target == 'ln_ufp_num_10s_ma_image_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    
    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.hdf5', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)


elif target == 'ln_ufp_num_10s_ma_spec_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')    
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.hdf5', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)
    
elif target == 'ln_ufp_size_10s_ma_image_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)
 
    
    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.hdf5', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)

elif target == 'ln_ufp_size_10s_ma_spec_label_raw':

    generator = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing, 
                                                     horizontal_flip=True,
                                                     vertical_flip = False)
    train_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='trn', [target, file]].reset_index(drop=True),  
                                                x_col= file, # 
                                                y_col= target,  # 
                                                class_mode = 'raw', 
                                                target_size=(256, 256), # all of our images will be resized to 256 x 256
                                                color_mode='rgb', 
                                                batch_size=64,
                                                shuffle=True)
    validate_generator = generator.flow_from_dataframe(dataframe=metadata.loc[metadata['set']=='val', [target, file]].reset_index(drop=True),
                                                     x_col= file,
                                                     y_col=target,
                                                     class_mode='raw',
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

    # Define callbacks
    csv_logger = K.callbacks.CSVLogger('./rishabh_model_files/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')
    model_checkpoint = K.callbacks.ModelCheckpoint('./rishabh_model_files/models/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.hdf5', monitor='val_loss', mode='auto', save_weights_only=False,save_best_only=True)
    reduce_lr_on_plateau = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, mode='auto', verbose=1)

else:
    print('!!!TYPO in exposure name') 

Current working directory: /home/scottweichenthal/Dropbox/IMAGINE Project/MSSI_Project
What do you want to model: ln_noise_10s_ma_spec_label_raw, ln_noise_10s_ma_image_label_raw, ln_ufp_num_10s_ma_spec_label_raw, ln_ufp_num_10s_ma_image_label_raw, ln_ufp_size_10s_ma_image_label_raw, ln_ufp_size_10s_ma_spec_label_raw ?ln_noise_10s_ma_image_label_raw
Found 345680 validated image filenames.
Found 22407 validated image filenames.


In [ ]:
#Train the model

# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

# Open a strategy scope.
with strategy.scope():
    model = get_compiled_model()
    

#Fit Model
model.fit(train_generator,
          validation_data=validate_generator,
          epochs=10, 
          steps_per_epoch=int(np.ceil(train_generator.samples/train_generator.batch_size)),
          validation_steps=int(np.ceil(validate_generator.samples/validate_generator.batch_size)),
          callbacks=[csv_logger, 
                     reduce_lr_on_plateau, 
                     model_checkpoint] 
         )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task

2024-05-13 17:35:05.360341: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 17:35:05.646613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-13 17:35:05.646879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-13 17:35:05.647109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/10
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1


2024-05-13 17:36:17.834012: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2024-05-13 17:36:18.658713: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2024-05-13 17:36:19.754882: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2024-05-13 17:36:19.770720: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2024-05-13 17:36:22.901021: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2024-05-13 17:36:22.901065: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if 

2889/5402 [===============>..............] - ETA: 36:40 - loss: 0.0445 - mae: 0.1390

2024-05-13 18:18:33.751894: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2024-05-13 18:18:33.844893: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performanc

4293/5402 [======================>.......] - ETA: 16:10 - loss: 0.0332 - mae: 0.1194

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3079/5402 [================>.............] - ETA: 33:56 - loss: 0.0069 - mae: 0.0656

In [50]:
#Identify Best Epoch
res = pd.read_csv(filepath_or_buffer='./model_development/logs/'+target+','+architecture+','+file+','+meta_data_name+'_Combined_IMAGINE_v2_GoPro.csv')
res = res.sort_values('val_loss', ascending=True).reset_index(drop=True)
# print(res)
best_epoch = res.epoch[0]
print(res.columns)



FileNotFoundError: [Errno 2] No such file or directory: './model_development/logs/ln_noise_10s_ma_spec_label_raw,ResNet50_linear_Nadam,image_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.csv'

In [ ]:
##### Generate Predictions for compiled data, add them to dataframe, save new dataframe #####

#Load Metadata
#metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_15012024.csv", low_memory=False)

#This version has GoPro data only in trn set (as opposed to random split between trn,val,tst)
#metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_04042024.csv", low_memory=False)
metadata = pd.read_csv("compiled data/metadata_goproUFP_V2_V3_04042024.csv", low_memory=False)

# check to see if each image file exists and remove rows from the results dataframe for which image files don't exist.
metadata['file_exists'] = metadata.apply(lambda row: os.path.isfile(row.image_path), axis = 1)

#Remove missing files from metadata to avoid errors in adding predictions to dataframe
metadata = metadata.loc[metadata['file_exists'] == True]
metadata.head(10)


In [ ]:
metadata.set.value_counts()

In [ ]:
#Generate Predictions and Add to Database

#Define Preprocessing functions
architecture_preprocessing_resnet50 = K.applications.resnet50.preprocess_input
architecture_preprocessing_xception = K.applications.xception.preprocess_input


#Define Test Generators
generator_tst_resnet50 = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing_resnet50, 
                                                     horizontal_flip=False,
                                                     vertical_flip = False)

generator_tst_xception = K.preprocessing.image.ImageDataGenerator(preprocessing_function=architecture_preprocessing_xception, 
                                                     horizontal_flip=False,
                                                     vertical_flip = False)

test_generator_resnet50_audio = generator_tst_resnet50.flow_from_dataframe(dataframe=metadata[['audio_path']].reset_index(drop=True),
                                                     x_col= 'audio_path',
                                                     class_mode= None,
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

test_generator_resnet50_image = generator_tst_resnet50.flow_from_dataframe(dataframe=metadata[['image_path']].reset_index(drop=True),
                                                     x_col= 'image_path',
                                                     class_mode= None,
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

test_generator_xception_audio = generator_tst_xception.flow_from_dataframe(dataframe=metadata[['audio_path']].reset_index(drop=True),
                                                     x_col= 'audio_path',
                                                     class_mode= None,
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

test_generator_xception_image = generator_tst_xception.flow_from_dataframe(dataframe=metadata[['image_path']].reset_index(drop=True),
                                                     x_col= 'image_path',
                                                     class_mode= None,
                                                     target_size=(256, 256),
                                                     color_mode='rgb',
                                                     batch_size=64,
                                                     shuffle=False)

                                                                       
######### - Generate Predictions for Best CNN Models - ##############



#Best model for log(ufp_num) based on images:  ln_ufp_num, image, ResNet50, raw
ln_ufp_num_ResNet50_image_raw_model = load_model('model_development/models/ln_ufp_num_10s_ma_image_label_raw,ResNet50_linear_Nadam,image_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.hdf5')
metadata['ln_ufp_num_ResNet50_image_raw_prediction'] = ln_ufp_num_ResNet50_image_raw_model.predict(x=test_generator_resnet50_image, 
                                                                                             steps=int(np.ceil(test_generator_resnet50_image.samples/test_generator_resnet50_image.batch_size)))
#Best model for log(ufp_num) based on audio: ln_ufp_num, audio, ResNet50, raw
ln_ufp_num_ResNet50_audio_raw_model = load_model('model_development/models/ln_ufp_num_10s_ma_spec_label_raw,ResNet50_linear_Nadam,audio_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.hdf5')
metadata['ln_ufp_num_ResNet50_audio_raw_prediction'] = ln_ufp_num_ResNet50_audio_raw_model.predict(x=test_generator_resnet50_audio, 
                                                                                             steps=int(np.ceil(test_generator_resnet50_audio.samples/test_generator_resnet50_audio.batch_size)))
#Best model for log(ufp_size) based on images: ln_ufp_size, image, ResNet50, raw
ln_ufp_size_ResNet50_image_raw_model = load_model('model_development/models/ln_ufp_size_10s_ma_image_label_raw,ResNet50_linear_Nadam,image_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.hdf5')
metadata['ln_ufp_size_ResNet50_image_raw_prediction'] = ln_ufp_size_ResNet50_image_raw_model.predict(x=test_generator_resnet50_image, 
                                                                                             steps=int(np.ceil(test_generator_resnet50_image.samples/test_generator_resnet50_image.batch_size)))
#Best model for log(ufp_size) based on audio: ln_ufp_size, audio, Xception, raw
ln_ufp_size_Xception_audio_raw_model = load_model('model_development/models/ln_ufp_size_10s_ma_spec_label_raw,Xception_linear_Nadam,audio_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.hdf5')
metadata['ln_ufp_size_Xception_audio_raw_prediction'] = ln_ufp_size_Xception_audio_raw_model.predict(x=test_generator_xception_audio,
                                                                                                     steps=int(np.ceil(test_generator_xception_audio.samples/test_generator_xception_audio.batch_size)))
#Best model for log(noise) based on images: ln_noise, image, ResNet50, raw
ln_noise_ResNet50_image_raw_model = load_model('model_development/models/ln_noise_10s_ma_image_label_raw,ResNet50_linear_Nadam,image_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.hdf5')
metadata['ln_noise_ResNet50_image_raw_prediction'] = ln_noise_ResNet50_image_raw_model.predict(x=test_generator_resnet50_image,
                                                                                               steps=int(np.ceil(test_generator_resnet50_image.samples/test_generator_resnet50_image.batch_size)))

#Best model for log(noise) based on audio: ln_noise, audio, Xception, raw
ln_noise_Xception_audio_raw_model = load_model('model_development/models/ln_noise_10s_ma_spec_label_raw,Xception_linear_Nadam,audio_path,GoPro_Training_Only_Combined_IMAGINE_v2_GoPro.hdf5')
metadata['ln_noise_Xception_audio_raw_prediction'] = ln_noise_Xception_audio_raw_model.predict(x=test_generator_xception_audio, 
                                                                                             steps=int(np.ceil(test_generator_xception_audio.samples/test_generator_xception_audio.batch_size)))               

#Save dataframe with all predictions added
metadata.to_csv(path_or_buf='compiled data/metadata_gopro_training_only_UFP_V2_V3_04042024_predictions_added.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt

# Extract data from DataFrame
epochs = res['epoch']
train_loss = res['loss']
val_loss = res['val_loss']

# Plotting loss curves
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()